In [1]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas


/home/ali/idrakai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

In [3]:
%ls data

data_idrak_dataset_train_df_.csv


In [4]:
df=pd.read_csv('data/data_idrak_dataset_train_df_.csv')

In [5]:
syn_aug = naw.SynonymAug(aug_src='wordnet')
ran_aug = naw.RandomWordAug(action="swap")
split_aug = naw.SplitAug()

In [45]:
aug_list={'syn_aug':syn_aug,'random_aug':ran_aug,'split_aug':split_aug}

In [59]:
def aumgentor(text,aug_list):
    augmented_text=[]
    for aug in aug_list:
        text_augmentor=aug_list[aug]
        augmented_text.append(text_augmentor.augment(text))
    return augmented_text

In [103]:
import numpy as np

In [123]:
def aug_text(aug_class_label='negative',val=100,aug_list={}):
    '''
       This function will augment texgt based on provided augmentor lists.
       Steps:
       1) Get a dataframe and extract the specified class from it
       2) Extract differnt Columns
       3) Itterate over each text and augment it 
       4) Make New Dataframe from Augmented
       5) Check for Column not present in augment 
       6) Randomly Droping rows
       7) Append with Parent dataframe
       8) Randomize
       
       augs:
           aug_class_labels: class to augment string
           val: how much samples to augment
           aug_list: list of augmentors
    '''
    try:
        temp_df=df[df['class_labels']==aug_class_label]
        class_labels=temp_df.iloc[0].class_labels
        class_number=temp_df.iloc[0]['class']
        texts=list(temp_df['cleaned_text'].values)
        augmented_texts=[]
        for text in texts:
            for aug in aug_list:
                augmented_texts+=aug_list[aug].augment(text)
        class_labels_list=len(augmented_texts)*[class_labels]
        class_numbers_list=len(augmented_texts)*[class_number]
        temp_df1=pd.DataFrame()
        temp_df1['class_labels']=class_labels_list
        temp_df1['class']=class_numbers_list
        temp_df1['cleaned_text']=augmented_texts
        
        #Check for columns whose are not in augmented dataframe
        labels_not_in_temp=list(set(temp_df.columns)-set(temp_df1.columns))
        #filling those columns rows with "Augmented Row" Text
        
        for label in labels_not_in_temp:
            temp_df1[label]=['augmented_row']*len(augmented_texts)
        print(len(temp_df1))
        if len(temp_df1)<=val:
            val=len(temp_df1)
        randomly_select=list(set(np.random.randint(len(temp_df1), size=val)))
        temp_df1=temp_df1.iloc[randomly_select]
        print(len(temp_df1))
        temp_df=temp_df.append(temp_df1)
    except Exception as e:
        print('An Error Occured \n {}'.format(e))
    

In [122]:
aug_text(aug_list=aug_list)

111
63


/tmp/ipykernel_5970/1081472915.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df=temp_df.append(temp_df1)


In [83]:
df1=pd.DataFrame(columns=['class','class_labels'])

In [84]:
list(set(df.columns)-set(df1.columns))

['cleaned_text', 'beam']

,class,class_labels
